### MRO 类和实例属性的查找顺序

**Python默认使用C3算法进行搜索**  

Python2.2中使用的经典类使用的是DFS方法进行搜索，例如下面这个继承链

In [7]:
class D:
    pass


class E:
    pass


# 子类B, C分别继承D, E
class B(D):
    pass


class C(E):
    pass


# 子类A1继承B, C
class A1(B, C):
    pass

在Python2.2中使用DFS的继承顺序是：A -> B -> D -> C -> E  
在菱形继承中，这种搜索算法会遗漏一些子类的属性，例如

In [8]:
# 菱形继承
class D:
    pass


# B, C都继承D
class B(D):
    pass


class C(D):
    pass


# A2继承B, C
class A2(B, C):
    pass

DFS搜索的继承链：A2 -> B -> D -> C  
此时如果C中重写了父类D的一个方法，调用该方法时  
按照继承链就只能搜索到父类D的方法返回，无法调用C重写后的方法  

BFS搜索也有类似的问题，在`A1`中如果使用BFS搜索，继承链为：A -> B -> C -> D -> E  
此时如果C，D中有重名的方法，按照这个搜索链，C的方法就会被先搜索到，从而无法使用D的方法，而C和D之间没有继承关系  
按照继承关系搜索应该是先在某个类及其父类中搜索后，再去别的继承链搜索（例如B -> D --> C -> E）  

Python3中使用的C3算法解决了上述问题，可以使用`__mro__`查看class的继承顺序

In [9]:
print(A1.__mro__)
print(A2.__mro__)

(<class '__main__.A1'>, <class '__main__.B'>, <class '__main__.D'>, <class '__main__.C'>, <class '__main__.E'>, <class 'object'>)
(<class '__main__.A2'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.D'>, <class 'object'>)


A1继承链：A1 -> B -> D -> C -> E -> Object  
A2继承链：A2 -> B -> C -> D -> Object  

Object是所有类的父类，Python3中所有类都会默认继承Object

#### super的调用顺序
**super不会直接调用最顶层的父类对象，而是按照MRO顺序调用其上一个直接父类对象**

In [10]:
class D:
    def __init__(self):
        print("D")


# 子类B, C继承D
class B(D):
    def __init__(self):
        print("B")
        super().__init__()


class C(D):
    def __init__(self):
        print("C")
        super().__init__()


# 子类A3继承B, C
class A3(B, C):
    def __init__(self):
        print("A3")
        super().__init__()

# 查看MRO
print(A3.__mro__)

(<class '__main__.A3'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.D'>, <class 'object'>)


In [9]:
# 查看调用结果
print(A3())

A3
B
C
D


可以看到，`super`的调用顺序遵循MRO顺序，A3的直接父类是B，B调用构造函数时不会直接调用顶层父类D的构造函数，而是按照MRO顺序调用C的构造函数